# Installing OpenCV

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


# Importing OpenCV

In [2]:
import cv2

# Exercise 1.1
This exercise aims to detect the number of cars on the main street.
## First implementation

In [3]:
def task1_original(video_file):
    status_list = [0, 0]
    car_count = 0
    initial_frame = None

    # Read video file
    video = cv2.VideoCapture(video_file)

    # Check if video file opened successfully
    if not video.isOpened(): 
        print('Error opening video file')

    # Get video duration in seconds
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps

    # Main loop to process each frame
    while True:
        # Read the current frame
        check, frame = video.read()

        # Continue processing if frame read successfully
        if check:
            status = 0

            # Extract the main street region from the frame
            height, width = frame.shape[:2]
            main_street = frame[int(height / 2):, :]

            # Convert the region to grayscale and apply Gaussian blur
            gray_frame = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
            blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

            # If it is the first frame, set it as the initial frame for comparison
            if initial_frame is None:
                initial_frame = blur_frame
                continue

            # Calculate the difference between the current frame and the initial frame
            delta_frame = cv2.absdiff(initial_frame, blur_frame)
            _, threshold_frame = cv2.threshold(delta_frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Find contours in the threshold frame
            contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rectangles = []

            # Iterate over the contours and extract bounding rectangles
            for c in contours:
                if cv2.contourArea(c) < 2000:
                    continue

                x, y, w, h = cv2.boundingRect(c)
                rectangles.append((x, y + int(height / 2), x + w, y + h + int(height / 2)))

            # Count rectangles and draw them on the frame
            for rectangle in rectangles:
                status += 1
                cv2.rectangle(frame, (rectangle[0], rectangle[1]), (rectangle[2], rectangle[3]), (0, 255, 0), 1)

            status_list.append(status)

            # Add to car count if current frame has more cars than previous frame
            if status_list[-1] > status_list[-2]:
                car_count += 1

            # Display frames
            cv2.imshow('Video', frame)
            cv2.imshow('Baseline image', initial_frame)
            cv2.imshow('Gray frame', gray_frame)
            cv2.imshow('Delta frame', delta_frame)   
            cv2.imshow('Threshold frame', threshold_frame)

            # Exit if 'q' key is pressed
            if cv2.waitKey(1) == ord('q'):
                break

        # Break if no more frames left
        else:
            break

    # Print the total number of cars and cars per minute
    print(f'Total number of cars: {car_count}')
    print(f'Cars per minute: {car_count / (duration / 60)}')

    # Release video object and close all windows
    video.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Traffic_Laramie_1.mp4

In [4]:
task1_original('Traffic_Laramie_1.mp4')

Total number of cars: 81
Cars per minute: 27.31564748201439


### Traffic_Laramie_2.mp4

In [5]:
task1_original('Traffic_Laramie_2.mp4')

Total number of cars: 92
Cars per minute: 52.23315669947009


## Second implementation

In [6]:
def task1_improved(video_file):
    status_list = [0, 0]
    car_count = 0
    initial_frame = None

    # Read video file
    video = cv2.VideoCapture(video_file)

    # Check if video file opened successfully
    if not video.isOpened(): 
        print('Error opening video file')

    # Get video duration in seconds
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps

    # Main loop to process each frame
    while True:
        # Read the current frame
        check, frame = video.read()

        # Continue processing if frame read successfully
        if check:
            status = 0

            # Extract the main street region from the frame
            height, width = frame.shape[:2]
            main_street = frame[int(height / 2):, :]

            # Convert the region to grayscale and apply Gaussian blur
            gray_frame = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
            blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

            # If it is the first frame, set it as the initial frame for comparison
            if initial_frame is None:
                initial_frame = blur_frame
                continue

            # Calculate the difference between the current frame and the initial frame
            delta_frame = cv2.absdiff(initial_frame, blur_frame)
            _, threshold_frame = cv2.threshold(delta_frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Find contours in the threshold frame
            contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rectangles = []

            # Iterate over the contours and extract bounding rectangles
            for c in contours:
                if cv2.contourArea(c) < 2000:
                    continue

                x, y, w, h = cv2.boundingRect(c)
                rectangles.append((x, y + int(height / 2), x + w, y + h + int(height / 2)))

            # Filter out overlapping rectangles
            extra_rectangles = []

            for i in range(len(rectangles)):
                for j in range(i + 1, len(rectangles)):
                    x1, y1, w1, h1 = rectangles[i]
                    x2, y2, w2, h2 = rectangles[j]

                    if (x1 >= x2 and x1 <= w2) or (w1 >= x2 and w1 <= w2) or (y1 >= y2 and y1 <= h2) or (h1 >= y2 and h1 <= h2):
                        extra_rectangles.append(rectangles[i])
                        break

            # Count non-overlapping rectangles and draw them on the frame
            for rectangle in rectangles:
                if rectangle not in extra_rectangles:
                    status += 1
                    cv2.rectangle(frame, (rectangle[0], rectangle[1]), (rectangle[2], rectangle[3]), (0, 255, 0), 1)

            status_list.append(status)

            # Add to car count if current frame has more cars than previous frame
            if status_list[-1] > status_list[-2]:
                car_count += 1

            # Display frames
            cv2.imshow('Video', frame)
            cv2.imshow('Baseline image', initial_frame)
            cv2.imshow('Gray frame', gray_frame)
            cv2.imshow('Delta frame', delta_frame)   
            cv2.imshow('Threshold frame', threshold_frame)

            # Exit if 'q' key is pressed
            if cv2.waitKey(1) == ord('q'):
                break

        # Break if no more frames left
        else:
            break

    # Print the total number of cars and cars per minute
    print(f'Total number of cars: {car_count}')
    print(f'Cars per minute: {car_count / (duration / 60)}')

    # Release video object and close all windows
    video.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Traffic_Laramie_1.mp4

In [7]:
task1_improved('Traffic_Laramie_1.mp4')

Total number of cars: 30
Cars per minute: 10.116906474820144


### Traffic_Laramie_2.mp4

In [8]:
task1_improved('Traffic_Laramie_2.mp4')

Total number of cars: 42
Cars per minute: 23.84557153671461


# Exercise 1.2
This exercise aims to detect the number of cars going towards the city centre.
## First implementation

In [9]:
def task2_original(video_file):
    status_list = [0, 0]
    car_count = 0
    initial_frame = None

    # Read video file
    video = cv2.VideoCapture(video_file)

    # Check if video file opened successfully
    if not video.isOpened(): 
        print('Error opening video file')

    # Get video duration in seconds
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps

    # Main loop to process each frame
    while True:
        # Read the current frame
        check, frame = video.read()

        # Continue processing if frame read successfully
        if check:
            status = 0

            # Extract the main street region from the frame
            height, width = frame.shape[:2]
            main_street = frame[int(height / 2):, :]

            # Convert the region to grayscale and apply Gaussian blur
            gray_frame = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
            blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

            # If it is the first frame, set it as the initial frame for comparison
            if initial_frame is None:
                initial_frame = blur_frame
                continue

            # Calculate the difference between the current frame and the initial frame
            delta_frame = cv2.absdiff(initial_frame, blur_frame)
            _, threshold_frame = cv2.threshold(delta_frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Find contours in the threshold frame
            contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rectangles = []

            # Iterate over the contours and extract bounding rectangles going towards city centre
            for c in contours:
                if cv2.contourArea(c) < 2000:
                    continue

                x, y, w, h = cv2.boundingRect(c)

                if (x + w <= int(width / 2) and y + h + int(height / 2) <= int(height / 4 * 3)):
                    rectangles.append((x, y + int(height / 2), x + w, y + h + int(height / 2)))

            # Count rectangles and draw them on the frame
            for rectangle in rectangles:
                status += 1
                cv2.rectangle(frame, (rectangle[0], rectangle[1]), (rectangle[2], rectangle[3]), (0, 255, 0), 1)

            status_list.append(status)

            # Add to car count if current frame has more cars than previous frame
            if status_list[-1] > status_list[-2]:
                car_count += 1

            # Display frames
            cv2.imshow('Video', frame)
            cv2.imshow('Baseline image', initial_frame)
            cv2.imshow('Gray frame', gray_frame)
            cv2.imshow('Delta frame', delta_frame)   
            cv2.imshow('Threshold frame', threshold_frame)

            # Exit if 'q' key is pressed
            if cv2.waitKey(1) == ord('q'):
                break

        # Break if no more frames left
        else:
            break

    # Print the total number of cars and cars per minute
    print(f'Total number of cars: {car_count}')
    print(f'Cars per minute: {car_count / (duration / 60)}')

    # Release video object and close all windows
    video.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Traffic_Laramie_1.mp4

In [10]:
task2_original('Traffic_Laramie_1.mp4')

Total number of cars: 17
Cars per minute: 5.732913669064748


### Traffic_Laramie_2.mp4

In [11]:
task2_original('Traffic_Laramie_2.mp4')

Total number of cars: 9
Cars per minute: 5.109765329295987


## Second implementation

In [12]:
def task2_improved(video_file):
    status_list = [0, 0]
    car_count = 0
    initial_frame = None

    # Read video file
    video = cv2.VideoCapture(video_file)

    # Check if video file opened successfully
    if not video.isOpened(): 
        print('Error opening video file')

    # Get video duration in seconds
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps

    # Main loop to process each frame
    while True:
        # Read the current frame
        check, frame = video.read()

        # Continue processing if frame read successfully
        if check:
            status = 0

            # Extract the main street region from the frame
            height, width = frame.shape[:2]
            main_street = frame[int(height / 2):, :]

            # Convert the region to grayscale and apply Gaussian blur
            gray_frame = cv2.cvtColor(main_street, cv2.COLOR_BGR2GRAY)
            blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

            # If it is the first frame, set it as the initial frame for comparison
            if initial_frame is None:
                initial_frame = blur_frame
                continue

            # Calculate the difference between the current frame and the initial frame
            delta_frame = cv2.absdiff(initial_frame, blur_frame)
            _, threshold_frame = cv2.threshold(delta_frame, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Find contours in the threshold frame
            contours, _ = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rectangles = []

            # Iterate over the contours and extract bounding rectangles going towards city centre
            for c in contours:
                if cv2.contourArea(c) < 2000:
                    continue

                x, y, w, h = cv2.boundingRect(c)

                if (x + w <= int(width / 2) and y + h + int(height / 2) <= int(height / 4 * 3)):
                    rectangles.append((x, y + int(height / 2), x + w, y + h + int(height / 2)))

            # Filter out overlapping rectangles
            extra_rectangles = []

            for i in range(len(rectangles)):
                for j in range(i + 1, len(rectangles)):
                    x1, y1, w1, h1 = rectangles[i]
                    x2, y2, w2, h2 = rectangles[j]

                    if (x1 >= x2 and x1 <= w2) or (w1 >= x2 and w1 <= w2) or (y1 >= y2 and y1 <= h2) or (h1 >= y2 and h1 <= h2):
                        extra_rectangles.append(rectangles[i])
                        break

            # Count non-overlapping rectangles and draw them on the frame
            for rectangle in rectangles:
                if rectangle not in extra_rectangles:
                    status += 1
                    cv2.rectangle(frame, (rectangle[0], rectangle[1]), (rectangle[2], rectangle[3]), (0, 255, 0), 1)

            status_list.append(status)

            # Add to car count if current frame has more cars than previous frame
            if status_list[-1] > status_list[-2]:
                car_count += 1

            # Display frames
            cv2.imshow('Video', frame)
            cv2.imshow('Baseline image', initial_frame)
            cv2.imshow('Gray frame', gray_frame)
            cv2.imshow('Delta frame', delta_frame)   
            cv2.imshow('Threshold frame', threshold_frame)

            # Exit if 'q' key is pressed
            if cv2.waitKey(1) == ord('q'):
                break

        # Break if no more frames left
        else:
            break

    # Print the total number of cars and cars per minute
    print(f'Total number of cars: {car_count}')
    print(f'Cars per minute: {car_count / (duration / 60)}')

    # Release video object and close all windows
    video.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Traffic_Laramie_1.mp4

In [13]:
task2_improved('Traffic_Laramie_1.mp4')

Total number of cars: 11
Cars per minute: 3.7095323741007196


### Traffic_Laramie_2.mp4

In [14]:
task2_improved('Traffic_Laramie_2.mp4')

Total number of cars: 9
Cars per minute: 5.109765329295987
